<h1>Notebook feta per Rubén Álvarez i Pol Pérez </h1>

In [1]:
%pip install -r requirements.txt --ignore-installed

^C
Note: you may need to restart the kernel to use updated packages.


<h1> <a href= https://gebakx.github.io/plh/s4/index.html#14>Pràctica 2.a: Detecció d'opinions (Supervised Learning) </a> </h1>

___

## Importacions i lectura del text inicial

In [ ]:
# Lectura de dades (Movie Reviews Corpus) NLTK

import nltk
import re
import sklearn
from sklearn.model_selection import train_test_split 
nltk.download('movie_reviews')
nltk.download('wordnet')
from nltk.corpus import movie_reviews as mr
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier


In [17]:
all_texts = []
all_categories = []

for fileid in mr.fileids():
    all_texts.append(mr.raw(fileid))
    all_categories.append(mr.categories(fileid)[0])

In [18]:
all_categories.count('neg')

1000

## Preprocessat

In [19]:
def preprocessing(text : str) -> str:
  """Preprocessat de text per a anàlisi de sentiment. Aplica:
      - Eliminació de dígits
      - Conversió a minúscules
      - Substitució de dos espais per un
      - Eliminació de tabs
      - Lematització

  Args:
      text (str): text to preprocess

  Returns:
      String: text preprocessed
  """

  for num in range(10):
    text = text.replace(str(num), "")

  text = text.lower()

  text = re.sub(" +", " ", text)

  text = text.replace("\n ", "\n  ")

  text = text.replace('\t','')

  lemmatizer = WordNetLemmatizer()
  text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
  
  return text

In [23]:
#Apliquem el preprocès a tots els textos

for text in all_texts:
  text = preprocessing(text)

plot : two teen couple go to a church party , drink and then drive . they get into an accident . one
the happy bastard's quick movie review damn that yk bug . it's got a head start in this movie starri
it is movie like these that make a jaded movie viewer thankful for the invention of the timex indigl
" quest for camelot " is warner bros . ' first feature-length , fully-animated attempt to steal clou
synopsis : a mentally unstable man undergoing psychotherapy save a boy from a potentially fatal acci
capsule : in on the planet mar police taking into custody an accused murderer face the title menace 
so ask yourself what " mm " ( " eight millimeter " ) is really all about . is it about a wholesome s
that's exactly how long the movie felt to me . there weren't even nine laugh in nine month . it's a 
call it a road trip for the walking wounded . stellan skarsg ? rd play such a convincingly zombified
plot : a young french boy see his parent killed before his eye by tim roth , oops . . . i m

## Definició i validació del model

### Separació train i test


In [ ]:
# Separem les dades en train i test
X_train, X_test, y_train, y_test = train_test_split(all_texts, all_categories, test_size=0.2, random_state=14)

### Aplicació Count Vectorizer

In [ ]:
# Crear un objecte CountVectorizer eliminant les stopwords
vectorizer = CountVectorizer(stop_words=stopwords.words('english'), min_df=5)

# Ajustar el vectoritzador a les dades d'entrenament
train_matrix = vectorizer.fit_transform(X_train)

# Transformar les dades de test
test_matrix = vectorizer.transform(X_test)

### Grid search cross validation (Random Forest)

In [ ]:
#Hyperparàmetres
n_estimators = [100, 200, 500]
max_depth = [3, 5, 10]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2]

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}

rf = RandomForestClassifier()
rf_cv = GridSearchCV(rf, param_grid, cv=5)
rf_cv.fit(train_matrix, y_train)
print("Tuned Random Forest Parameters: {}".format(rf_cv.best_params_))

Tuned Random Forest Parameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 500}


### Random Forest amb els millors paràmetres

In [ ]:
# Random forest
from sklearn.ensemble import RandomForestClassifier

# Crear un objecte RandomForestClassifier per saber si el text és positiu o negatiu
classifier = RandomForestClassifier(random_state=14, n_estimators=rf_cv.best_params_['n_estimators'], max_depth=rf_cv.best_params_['max_depth'], min_samples_split=rf_cv.best_params_['min_samples_split'], min_samples_leaf=rf_cv.best_params_['min_samples_leaf'])

# Entrenar el classificador
classifier.fit(train_matrix, y_train)

# Predir las categories de les dades de validació
test_predictions = classifier.predict(test_matrix)

## Resultats (aplicant el model a les dades test)

In [ ]:
for i, prediction in enumerate(test_predictions):
    print(f"Prediction: {prediction}")
    print(f"Real_category: {y_test[i]}")
    print("-----------------------------")
    print(f"Text: {X_test[i][:100]}")

Prediction: neg
Real_category: pos
-----------------------------
Text: when you go to the movies as much as i do , you unfortunately end up seeing certain movie trailers o
Prediction: pos
Real_category: pos
-----------------------------
Text: hollywood has really done the whole dracula thing to death ( no pun intended - honest ) . 
it's a sh
Prediction: neg
Real_category: neg
-----------------------------
Text: i never understood what the clich ? " hell on earth " truly meant until very recently . 
i've just n
Prediction: neg
Real_category: pos
-----------------------------
Text: whenever studio executives try to combine the successful formula of two blockbuster films , the end 
Prediction: neg
Real_category: neg
-----------------------------
Text:  " return to horror high , " wants to be a couple different types of movies at once . 
the film tell
Prediction: pos
Real_category: neg
-----------------------------
Text: don't get me wrong -- i tend to appreciate besson's naive , unashamed

In [ ]:
# Calcular la precisión
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, test_predictions)

accuracy

0.8425

## Predicció per un text importat de fora

In [ ]:
def predict_sentiment(text):
  """Predicts the sentiment of a text

  Args:
      text (str): text to predict

  Returns:
      String: sentiment prediction
  """
  
  text = preprocessing(text)
  text_matrix = vectorizer.transform([text])
  prediction = classifier.predict(text_matrix)
  return prediction[0]

In [ ]:
predict_sentiment("""A world, by the way, systemic, where the physics and artificial intelligence of the characters and enemies generate a lot of spontaneous moments. Especially in the combats, constant and hilarious, especially when giant monsters are involved; and very customizable thanks to the great vocation and pawn systems. We must point out the performance flaws, a somewhat limited bestiary and a visual section with its lights and shadows, but in the end they are minor details in a title that has made us say out loud several times "I can't believe it." Such a surprising title that teaches you to expect the unexpected.""")

'pos'

<h1> <a href= https://gebakx.github.io/plh/s4/index.html#17> Pràctica 2.b: Detecció d'opinions (Unsupervised Learning) </a> </h1>

___

## Importacions i lectura del text inicial

In [29]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.wsd import lesk
import re
from sklearn.model_selection import train_test_split 

nltk.download('movie_reviews')
nltk.download('wordnet')
from nltk.corpus import movie_reviews as mr
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn


[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Pol\AppData\Roaming\nltk_data...


[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Pol\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [30]:
all_texts = []
all_categories = []

for fileid in mr.fileids():
    all_texts.append(mr.raw(fileid))
    all_categories.append(mr.categories(fileid)[0])

## Preprocessat

In [31]:
def preprocessing(text):
  """Preprocessat de text per a anàlisi de sentiment. Aplica:
      - Eliminació de dígits
      - Conversió a minúscules
      - Substitució de dos espais per un
      - Eliminació de tabs
      - Tokenització

  Args:
      text (str): text to preprocess

  Returns:
      String: text preprocessed
  """
  
  for num in range(10):
    text = text.replace(str(num), "")

  text = text.lower()

  text = re.sub(" +", " ", text)

  text = text.replace("\n ", "\n  ")

  text = text.replace('\t','')

  text = nltk.word_tokenize(text)
  
  return text

In [32]:
all_preprocesed_texts = []

for text in all_texts:
  text = preprocessing(text)
  all_preprocesed_texts.append(text)

In [33]:
print(all_preprocesed_texts[0][0:10])

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party']


## Desambiguació amb Lesk

In [38]:
for text in all_preprocesed_texts:
    text_pos_score = 1
    text_neg_score = 1
    for word in text:
        synset = lesk(text, word)
        if synset:
            synset_name = synset.name()
            senti_synset = swn.senti_synset(synset_name)
            word_pos_score = senti_synset.pos_score()
            word_neg_score = senti_synset.neg_score()
            if word_pos_score != 0:
                text_pos_score *= word_pos_score
            if word_neg_score != 0:
                text_neg_score *= word_neg_score
    print(text_pos_score, text_neg_score)



7.556340577003996e-61 6.56549556697075e-64
9.81469183839774e-20 3.497202527569243e-15
6.639130885048938e-53 5.451286748243878e-54
3.5084527809781232e-50 1.5522719523894373e-41
8.715087448897468e-71 8.213323506026282e-91
3.387651502759877e-65 1.0588831250410424e-68
2.7256433741219392e-58 7.496369620043646e-60
2.5320485570595054e-53 5.31851763099776e-44
2.964277158401304e-53 3.8615588052699875e-57
5.256597935806597e-49 3.285483976559769e-64
1.236475869554304e-66 2.6986930632157127e-59
6.778492765791529e-35 3.455163459213773e-27
1.7749370367472766e-27 5.857956711177864e-28
4.60768894641941e-72 2.2809277997315136e-73
8.420791648051733e-39 2.652050783588981e-45
3.636266880758033e-42 6.215744024036675e-42
4.395917633783864e-56 3.659820900383227e-43
4.234682005238138e-65 1.0311760047507055e-63
2.3961649996088234e-27 1.6675961149069038e-16
8.421978504328807e-66 8.499285283496198e-70
2.8702268953619592e-52 3.660438404557174e-66
3.5468926770533086e-31 2.9788298287169804e-34
2.566698210843065e-62

KeyboardInterrupt: 